In [ ]:
imUsingColab = False
shouldDownload = True # Set this to False after the first run to avoid redownloading the dataset
if shouldDownload:
  if imUsingColab:
    # Generate the .env file within the colab environment programatically
    file = open('.env','w') 
    file.write('KAGGLE_USERNAME=') # Populate this with your KAGGLE_USERNAME DO NOT COMMIT!!
    file.write('\nKAGGLE_KEY=') # Populate this with your KAGGLE api Key DO NOT COMMIT!!
    file.close()
    # Fetch the downloadData dependency 
    import urllib.request
    urllib.request.urlretrieve ("https://raw.githubusercontent.com/lihaojin/Movie_Data_Analysis/master/downloadData.py", "downloadData.py")

  import downloadData
  downloadData.download(imUsingColab)


In [ ]:
%run datasets.py
%run functions.py

import pandas as pd
import numpy as np
import math, nltk, warnings
from nltk.corpus import wordnet
from sklearn import linear_model
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz

warnings.filterwarnings('ignore')

In [ ]:
df = mergeData()
df.head()

In [ ]:
set_keywords = set()
for list_keywords in df['keywords'].str.split('|').values:
    if isinstance(list_keywords, float): continue  # only happen if list_keywords = NaN
    set_keywords = set_keywords.union(list_keywords)

In [ ]:
keyword_occurences, dum = count_word(df, 'keywords', set_keywords)
keyword_occurences[1:6]

In [ ]:
df['genres'] = df['genres'].astype(str)
genre_labels = set()
for s in df['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

In [ ]:
keyword_occurences, dum = count_word(df, 'genres', genre_labels)
keyword_occurences[:5]

In [ ]:
#doubled_entries = df_initial[df_initial.duplicated()]
doubled_entries = df[df.movieId.duplicated()]
doubled_entries.shape

In [ ]:
df_temp = df

In [ ]:
list_var_duplicates = ['title', 'year', 'directors']

In [ ]:
list_duplicates = df_temp['title'].map(df_temp['title'].value_counts() > 1)
print("Nb. of duplicate entries: {}".format(
    len(df_temp[list_duplicates][list_var_duplicates])))

In [ ]:
#df_temp[list_duplicates][list_var_duplicates].sort_values('movie_title')[31:41]
df_temp[list_duplicates][list_var_duplicates].sort_values('title')

In [ ]:
keywords, keywords_roots, keywords_select = keywords_inventory(df, column = 'keywords')

In [ ]:
# Plot of a sample of keywords that appear in close varieties 
#------------------------------------------------------------
icount = 0
for s in keywords_roots.keys():
    if len(keywords_roots[s]) > 1: 
        icount += 1
        if icount < 15: print(icount, keywords_roots[s], len(keywords_roots[s]))

In [ ]:
df_keywords_cleaned = replacement_df_keywords(df, keywords_select,
                                               roots = True)

In [ ]:
# Count of the keywords occurences
#----------------------------------
keyword_occurences, keywords_count = count_word(df_keywords_cleaned,'keywords',keywords)
keyword_occurences[1:6]

In [ ]:
# Exemple of a list of synonyms given by NLTK
#---------------------------------------------------
mot_cle = 'alien'
lemma = get_synonyms(mot_cle)
for s in lemma:
    print(' "{:<30}" in keywords list -> {} {}'.format(s, s in keywords,
                                                keywords_count[s] if s in keywords else 0 ))

In [ ]:
keyword_occurences.sort(key = lambda x:x[1], reverse = False)
key_count = dict()
for s in keyword_occurences:
    key_count[s[0]] = s[1]
#__________________________________________________________________________
# Creation of a dictionary to replace keywords by higher frequency keywords
replacement_mot = dict()
icount = 0
for index, [mot, nb_apparitions] in enumerate(keyword_occurences):
    if nb_apparitions > 5: continue  # only the keywords that appear less than 5 times
    lemma = get_synonyms(mot)
    if len(lemma) == 0: continue     # case of the plurals
    #_________________________________________________________________
    list_mots = [(s, key_count[s]) for s in lemma 
                  if test_keyword(s, key_count, key_count[mot])]
    list_mots.sort(key = lambda x:(x[1],x[0]), reverse = True)    
    if len(list_mots) <= 1: continue       # no replacement
    if mot == list_mots[0][0]: continue    # replacement by himself
    icount += 1
    if  icount < 8:
        print('{:<12} -> {:<12} (init: {})'.format(mot, list_mots[0][0], list_mots))    
    replacement_mot[mot] = list_mots[0][0]

print(90*'_'+'\n'+'The replacement concerns {}% of the keywords.'
      .format(round(len(replacement_mot)/len(keywords)*100,2)))

In [ ]:
# 2 successive replacements
#---------------------------
print('Keywords that appear both in keys and values:'.upper()+'\n'+45*'-')
icount = 0
for s in replacement_mot.values():
    if s in replacement_mot.keys():
        icount += 1
        if icount < 10: print('{:<20} -> {:<20}'.format(s, replacement_mot[s]))

for key, value in replacement_mot.items():
    if value in replacement_mot.keys():
        replacement_mot[key] = replacement_mot[value]          

In [ ]:
df_keywords_synonyms = \
            replacement_df_keywords(df_keywords_cleaned, replacement_mot, roots = False)   
keywords, keywords_roots, keywords_select = \
            keywords_inventory(df_keywords_synonyms, column = 'keywords')

In [ ]:
new_keyword_occurences, keywords_count = count_word(df_keywords_synonyms,
                                                    'keywords',keywords)
new_keyword_occurences[1:6]

In [ ]:
df_keywords_occurence = \
    replacement_df_low_frequency_keywords(df_keywords_synonyms, new_keyword_occurences)
keywords, keywords_roots, keywords_select = \
    keywords_inventory(df_keywords_occurence, column = 'keywords')   

In [ ]:
#-------------------
new_keyword_occurences, keywords_count = count_word(df_keywords_occurence,
                                                    'keywords',keywords)
new_keyword_occurences[1:6]

In [ ]:
dum = find_similarities(df, 0, del_sequels = True, verbose = True)